In [1]:
pip install llama-cpp-python


Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
from tensorflow import keras
import os
import string
import chromadb
import warnings
warnings.filterwarnings('ignore')
from transformers import pipeline
from llama_cpp import Llama


2024-10-15 09:05:13.980846: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-15 09:05:14.005677: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 09:05:14.030590: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 09:05:14.039068: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 09:05:14.062929: I tensorflow/core/platform/cpu_feature_guar

In [3]:
documents = []
metadata = []
ids = []

with open('../data/spells_english.txt', 'r', encoding='utf-8') as spells:
    for line in spells:
        line=line.split(" - ") #spells are separated by their description by a " - "
        metadata.append({'spell': line[0]}) #use the spell name as metadata. ChromaDB requires a dictionary here
        documents.append(line[1]) #use the description as documents
        ids.append(line[0]) #we also need an ID so use the spell name again.
print(ids)

['Aberto', 'Accio', 'Aguamenti', 'Alohomora', 'Anapneo', 'Aparecium', 'Apparate', 'Ascendio', 'Avada Kedavra', 'Avis', 'Bat-Bogey Hex', 'Bombardo', 'Brackium Emendo', 'Capacious Extremis', 'Confundo', 'Conjunctivitis Curse', 'Crinus Muto', 'Crucio', 'Diffindo', 'Disillusionment Charm', 'Disapparate', 'Engorgio', 'Episkey', 'Expecto patronum', 'Erecto', 'Evanesco', 'Expelliarmus', 'Ferula', 'Fidelius Charm', 'Fiendfyre Curse', 'Finite Incantatem', 'Furnunculus Curse', 'Geminio', 'Glisseo', 'Homenum Revelio', 'Homonculus Charm', 'Immobulus', 'Impedimenta', 'Incarcerous', '*Imperio', 'Impervius', 'Incendio', 'Langlock', 'Legilimens', 'Levicorpus', 'Locomotor Mortis', 'Lumos', '*Morsmordre', 'Mucus Ad Nauseam', 'Muffliato', 'Nox', 'Obliviate', 'Obscuro', 'Oculus Reparo', 'Oppugno', 'Petrificus Totalus', 'Periculum', 'Piertotum Locomotor', 'Protean Charm', 'Protego', 'Reducto', 'Reducio', 'Renneverate', 'Reparifors', 'Reparo', 'Rictusempra', 'Riddikulus', 'Scourgify', '*Sectumsempra', 'Serp

In [4]:

client = chromadb.Client()

# create or open collection named 'spells' 
spells_collection = client.get_or_create_collection("spells")

# add spells to the collection, ignored for ids that already exist 
spells_collection.add(
    documents=documents,
    metadatas=metadata,
    ids=ids
)

In [5]:
user_input="How do I create light"
results = spells_collection.query(
    query_texts=[user_input],
    n_results=1
)
results

{'ids': [['Lumos']],
 'distances': [[1.327547311782837]],
 'metadatas': [[{'spell': 'Lumos'}]],
 'embeddings': None,
 'documents': [["Illuminates the caster's wand\n"]],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [6]:
result=results['metadatas'][0][0]['spell']+" - " + results['documents'][0][0][:-1]
result

"Lumos - Illuminates the caster's wand"